In [1]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.3 MB/s eta 0:00:00


In [2]:
import ast
import json
import faiss
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

In [3]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def get_embeddings_file_path(repo_name):
    repo = "_".join(repo_name.split("/"))
    embeddings_file_path = f'/content/drive/My Drive/NLP/Project/data_new/embeddings/{repo}_files_infos.csv'
    return embeddings_file_path

In [5]:
llm_results_file_path = f'/content/drive/My Drive/NLP/Project/data_new/results.jsonl'
swebench_file_path = f'/content/drive/My Drive/NLP/Project/data_new/swebench_updated.csv'

In [6]:
swebench_df = pd.read_csv(swebench_file_path)
swebench_df

,repo,base_commit,modified_files,problem_statement,hints_text,files_in_repo,hashes
0,astropy/astropy,d16bfe05a744909de4b27f5875fe0d4ed41ce607,['astropy/modeling/separable.py'],Modeling's `separability_matrix` does not comp...,NaN,"['setup.py', 'conftest.py', 'astropy/logger.py...",['f3938f6642f4286182c4254164d30ea344da1f57586d...
1,astropy/astropy,298ccb478e6bf092953bca67a3d29dc6c35f6752,['astropy/timeseries/core.py'],TimeSeries: misleading exception when required...,The relevant code that produces the misleading...,"['setup.py', 'conftest.py', 'astropy/logger.py...",['f3938f6642f4286182c4254164d30ea344da1f57586d...
2,astropy/astropy,6ed769d58d89380ebaa1ef52b300691eefda8928,['astropy/table/table.py'],Consider removing auto-transform of structured...,@mhvk - I'm happy to do this PR if you think i...,"['setup.py', 'conftest.py', 'astropy/logger.py...",['f3938f6642f4286182c4254164d30ea344da1f57586d...
3,astropy/astropy,6500928dc0e57be8f06d1162eacc3ba5e2eff692,['astropy/coordinates/builtin_frames/__init__....,A direct approach to ITRS to Observed transfor...,"cc @StuartLittlefair, @adrn, @eteq, @eerovaher...","['setup.py', 'conftest.py', 'astropy/logger.py...",['f3938f6642f4286182c4254164d30ea344da1f57586d...
4,astropy/astropy,19cc80471739bcb67b7e8099246b391c355023ee,['astropy/io/ascii/html.py'],ASCII table output to HTML does not support su...,Welcome to Astropy 👋 and thank you for your fi...,"['setup.py', 'conftest.py', 'astropy/logger.py...",['f3938f6642f4286182c4254164d30ea344da1f57586d...
...,...,...,...,...,...,...,...
495,sympy/sympy,e8c22f6eac7314be8d92590bfff92ced79ee03e2,['sympy/physics/units/unitsystem.py'],collect_factor_and_dimension does not detect e...,NaN,"['setup.py', 'setupegg.py', 'conftest.py', 'is...",['bf6340d7ca6c6164d67756bfeb625659e294c380bdbd...
496,sympy/sympy,809c53c077485ca48a206cee78340389cb83b7f1,['sympy/combinatorics/homomorphisms.py'],`_check_homomorphism` is broken on Permutation...,NaN,"['setup.py', 'setupegg.py', 'conftest.py', 'is...",['e18ef7e45477477cfdc9e4d1642c9d9b3e002a90371f...
497,sympy/sympy,193e3825645d93c73e31cdceb6d742cc6919624d,['sympy/polys/rings.py'],`PolyElement.as_expr()` not accepting symbols\...,NaN,"['setup.py', 'setupegg.py', 'conftest.py', 'is...",['e18ef7e45477477cfdc9e4d1642c9d9b3e002a90371f...
498,sympy/sympy,b1cb676cf92dd1a48365b731979833375b188bf2,['sympy/core/numbers.py'],"Rational calc value error\npython 3.11, sympy ...",This should probably raise an error. The expec...,"['setup.py', 'setupegg.py', 'conftest.py', 'is...",['e18ef7e45477477cfdc9e4d1642c9d9b3e002a90371f...


In [7]:
ground_truth = list(swebench_df['modified_files'].apply(ast.literal_eval))

# LLM Results

In [8]:
# Initialize an empty list to store the JSON objects
llm_results = []
# Open the jsonl file for reading
with open(llm_results_file_path, 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append it to the list
        llm_results.append(json.loads(line.strip()))
# Now json_objects contains all the parsed JSON objects from the file
print(len(llm_results))

500


In [9]:
# top_1_results = []
# top_2_results = []
# top_3_results = []
llm_predictions = []
for res in llm_results:
    files = []
    if res['answered']:
        for i in range(1,res['num_files']+1):
            files.append(res['files'][f'rank_{i}'])
    llm_predictions.append(files)
    # top_1_results.append(files[:1])
    # top_2_results.append(files[:2])
    # top_3_results.append(files[:3])

In [10]:
print(len(llm_results), len(llm_predictions))
# print(len(results), len(predictions), len(top_1_results), len(top_2_results), len(top_3_results))

500 500


In [19]:
def get_metrics_for_instance(ground_truth, pred, k):
    count = 0
    for i in ground_truth:
        if i in pred[:k]:
            count+=1
    precision = count/k if len(ground_truth)>k else count/len(ground_truth)
    recall = count/len(ground_truth)
    return (recall,precision)

In [20]:
def compute_metrics(ground_truth, predictions, k_values):
    recall_scores = {k: [] for k in k_values}
    precision_scores = {k: [] for k in k_values}
    for gt,pred in zip(ground_truth, predictions):
        for k in k_values:
            recall,precision = get_metrics_for_instance(gt, pred, k)
            recall_scores[k].append(recall)
            precision_scores[k].append(precision)
    # print("Precision_scores",precision_scores)
    # print("Recall_scores",recall_scores)
    results = {}
    for k in recall_scores:
        results[f"recall@{k}"] = np.mean(recall_scores[k])
        results[f"precision@{k}"] = np.mean(precision_scores[k])
    return results

In [13]:
print(ground_truth)

[['astropy/modeling/separable.py'], ['astropy/timeseries/core.py'], ['astropy/table/table.py'], ['astropy/coordinates/builtin_frames/__init__.py', 'astropy/coordinates/builtin_frames/intermediate_rotation_transforms.py', 'astropy/coordinates/builtin_frames/itrs.py', 'astropy/coordinates/builtin_frames/itrs_observed_transforms.py'], ['astropy/io/ascii/html.py'], ['astropy/wcs/wcsapi/wrappers/sliced_wcs.py'], ['astropy/units/quantity.py'], ['astropy/coordinates/sky_coordinate.py'], ['astropy/io/ascii/rst.py'], ['astropy/io/fits/connect.py'], ['astropy/io/ascii/qdp.py'], ['astropy/units/format/cds.py', 'astropy/units/format/cds_parsetab.py'], ['astropy/io/fits/card.py'], ['astropy/io/fits/diff.py'], ['astropy/io/fits/card.py'], ['astropy/nddata/mixins/ndarithmetic.py'], ['astropy/utils/misc.py'], ['astropy/units/decorators.py'], ['astropy/units/core.py'], ['astropy/utils/introspection.py'], ['astropy/io/fits/card.py', 'astropy/io/fits/header.py'], ['astropy/units/quantity.py'], ['django/c

In [14]:
print(llm_predictions)

[['astropy/modeling/separable.py', 'astropy/modeling/__init__.py'], ['astropy/timeseries/core.py'], ['astropy/table/table.py'], ['astropy/coordinates/transformations.py', 'astropy/coordinates/baseframe.py', 'astropy/coordinates/itrs.py', 'astropy/coordinates/altaz.py', 'astropy/coordinates/hadec.py'], ['astropy/io/ascii/html.py', 'astropy/io/ascii/core.py'], ['astropy/wcs/wcsapi/wrappers/sliced_wcs.py'], ['astropy/units/quantity.py'], ['astropy/coordinates/sky_coordinate.py'], ['astropy/io/ascii/ui.py', 'astropy/io/ascii/core.py', 'astropy/io/registry/core.py'], ['astropy/io/registry/base.py'], ['astropy/io/ascii/qdp.py'], ['astropy/units/format/cds.py'], ['astropy/io/fits/card.py'], ['astropy/io/fits/diff.py'], ['astropy/io/fits/card.py', 'astropy/io/fits/header.py'], ['astropy/nddata/nddata.py'], ['astropy/utils/inspect.py', 'astropy/tests/tests/test_run_tests.py'], ['astropy/utils/decorators.py'], ['astropy/units/core.py'], ['astropy/utils/introspection.py'], ['astropy/io/fits/heade

In [15]:
print("Swebench LLM Basic Query Results : ")
res = compute_metrics(ground_truth, llm_predictions, [1,2,3])
for key in res:
    print(f"{key} : {res[key]}")

Swebench LLM Basic Query Results : 
recall@1 : 0.5893285714285714
precision@1 : 0.642
recall@2 : 0.6615619047619048
precision@2 : 0.67
recall@3 : 0.6725619047619048
precision@3 : 0.6753333333333332


# Semantic Similarity (Vector similarity)

In [8]:
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the UniXcoder model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/unixcoder-base")
model = AutoModel.from_pretrained("microsoft/unixcoder-base").to(device)  # Move the model to GPU if available

def get_embeddings(text: str):
    # Tokenize the input text and convert it to input IDs
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # Move the tokenized inputs to GPU if available
    inputs = {key: value.to(device) for key, value in inputs.items()}
    # Get the embeddings from the model
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Averaging the token embeddings
    return embeddings

def aggregate_embeddings(embeddings):
    # Aggregate embeddings from all chunks by averaging
    embeddings_tensor = torch.cat(embeddings, dim=0)
    aggregated_embedding = embeddings_tensor.mean(dim=0, keepdim=True)
    return aggregated_embedding

def get_document_embedding(document: str, chunk_size=512):
    # Split the document into smaller chunks if it exceeds the chunk size (token limit)
    chunks = [document[i:i + chunk_size] for i in range(0, len(document), chunk_size)]
    embeddings = []
    for chunk in chunks:
        embeddings.append(get_embeddings(chunk))
    final_embedding = aggregate_embeddings(embeddings)
    return final_embedding.squeeze().cpu().numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/938k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/444k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

In [9]:
def search_top_k_docs(file_embeddings, query_embedding, k=15):
    # index_cpu = faiss.IndexFlatL2(768)
    index_cpu = faiss.IndexFlatIP(768)
    faiss.normalize_L2(file_embeddings)
    faiss.normalize_L2(query_embedding)
    index_cpu.add(file_embeddings)
    if device=='cuda':
        gpu_res = faiss.StandardGpuResources()
        index_gpu = faiss.index_cpu_to_gpu(gpu_res, 0, index_cpu)
        distances, indices = index_gpu.search(query_embedding, k)
    else:
        distances, indices = index_cpu.search(query_embedding, k)
    return distances[0], indices[0]

In [23]:
semantic_search_predictions = []
for idx,row in swebench_df.iterrows():
    repo = row["repo"]
    hashes = ast.literal_eval(row["hashes"])
    embeddings_df = pd.read_csv(get_embeddings_file_path(repo))
    associated_rows = embeddings_df[embeddings_df['hash'].isin(hashes)]
    embeddings = associated_rows['embeddings'].apply(ast.literal_eval)
    embeddings = embeddings.apply(lambda x: [float(i) for i in x])
    embeddings = np.array(embeddings.tolist(),  dtype='float32')
    file_paths = associated_rows['file_path'].tolist()
    query_embedding = get_document_embedding(f"{row['problem_statement']} {row['hints_text']}")
    query_embedding = np.expand_dims(query_embedding, axis=0)
    dist, indices = search_top_k_docs(embeddings, query_embedding, 15)
    res_files = [file_paths[i] for i in indices]
    semantic_search_predictions.append(res_files)
    print(f"Processed row {idx}")

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/NLP/Project/data_new/embeddings/scikit-learn_scikit-learn_files_infos.csv'

In [10]:
semantic_search_predictions = []
swebench_df2 = swebench_df[349:500]

In [11]:
for idx,row in swebench_df2.iterrows():
    repo = row["repo"]
    hashes = ast.literal_eval(row["hashes"])
    embeddings_df = pd.read_csv(get_embeddings_file_path(repo))
    associated_rows = embeddings_df[embeddings_df['hash'].isin(hashes)]
    embeddings = associated_rows['embeddings'].apply(ast.literal_eval)
    embeddings = embeddings.apply(lambda x: [float(i) for i in x])
    embeddings = np.array(embeddings.tolist(),  dtype='float32')
    file_paths = associated_rows['file_path'].tolist()
    query_embedding = get_document_embedding(f"{row['problem_statement']} {row['hints_text']}")
    query_embedding = np.expand_dims(query_embedding, axis=0)
    dist, indices = search_top_k_docs(embeddings, query_embedding, 15)
    res_files = [file_paths[i] for i in indices]
    semantic_search_predictions.append(res_files)
    print(f"Processed row {idx}")

Processed row 349
Processed row 350
Processed row 351
Processed row 352
Processed row 353
Processed row 354
Processed row 355
Processed row 356
Processed row 357
Processed row 358
Processed row 359
Processed row 360
Processed row 361
Processed row 362
Processed row 363
Processed row 364
Processed row 365
Processed row 366
Processed row 367
Processed row 368
Processed row 369
Processed row 370
Processed row 371
Processed row 372
Processed row 373
Processed row 374
Processed row 375
Processed row 376
Processed row 377
Processed row 378
Processed row 379
Processed row 380
Processed row 381
Processed row 382
Processed row 383
Processed row 384
Processed row 385
Processed row 386
Processed row 387
Processed row 388
Processed row 389
Processed row 390
Processed row 391
Processed row 392
Processed row 393
Processed row 394
Processed row 395
Processed row 396
Processed row 397
Processed row 398
Processed row 399
Processed row 400
Processed row 401
Processed row 402
Processed row 403
Processed 

In [12]:
new_df = swebench_df[['modified_files']].copy()[349:500]
new_df['predicted_files'] = semantic_search_predictions
new_df.to_csv('/content/drive/My Drive/NLP/Project/data_new/semantic_search_res2.csv', index=False)

In [13]:
len(semantic_search_predictions)

151

In [14]:
new_df1 = pd.read_csv('/content/drive/My Drive/NLP/Project/data_new/semantic_search_res1.csv')
new_df1

,modified_files,predicted_files
0,['astropy/modeling/separable.py'],"['astropy/modeling/tests/test_separable.py', '..."
1,['astropy/timeseries/core.py'],"['docs/conf.py', 'astropy/conftest.py', 'astro..."
2,['astropy/table/table.py'],"['astropy/table/serialize.py', 'astropy/table/..."
3,['astropy/coordinates/builtin_frames/__init__....,['astropy/coordinates/builtin_frames/utils.py'...
4,['astropy/io/ascii/html.py'],"['astropy/io/ascii/tests/test_write.py', 'docs..."
...,...,...
344,['src/_pytest/skipping.py'],"['doc/en/conf.py', 'src/pytest/__init__.py', '..."
345,['src/_pytest/capture.py'],"['testing/test_terminal.py', 'testing/test_doc..."
346,['src/_pytest/logging.py'],"['testing/logging/test_fixture.py', 'testing/l..."
347,['src/_pytest/pathlib.py'],"['doc/en/conf.py', 'testing/test_error_diffs.p..."


In [15]:
new_df2 = pd.read_csv('/content/drive/My Drive/NLP/Project/data_new/semantic_search_res2.csv')
new_df2

,modified_files,predicted_files
0,['sklearn/linear_model/ridge.py'],"['sklearn/linear_model/ridge.py', 'examples/sv..."
1,['sklearn/metrics/cluster/supervised.py'],['examples/calibration/plot_compare_calibratio...
2,['sklearn/feature_extraction/text.py'],['sklearn/feature_extraction/tests/test_text.p...
3,['sklearn/model_selection/_search.py'],['examples/model_selection/plot_randomized_sea...
4,['sklearn/linear_model/logistic.py'],['sklearn/linear_model/tests/test_logistic.py'...
...,...,...
146,['sympy/physics/units/unitsystem.py'],['sympy/physics/units/tests/test_quantities.py...
147,['sympy/combinatorics/homomorphisms.py'],['sympy/combinatorics/tests/test_homomorphisms...
148,['sympy/polys/rings.py'],"['sympy/printing/tests/test_repr.py', 'sympy/p..."
149,['sympy/core/numbers.py'],"['sympy/core/numbers.py', 'sympy/external/pyth..."


In [16]:
ss_predictions = []
ss_pred_1 = new_df1['predicted_files'].apply(ast.literal_eval).to_list()
ss_pred_2 = new_df2['predicted_files'].apply(ast.literal_eval).to_list()
ss_predictions.extend(ss_pred_1)
ss_predictions.extend(ss_pred_2)
print(len(ss_predictions))

500


In [17]:
ss_predictions

[['astropy/modeling/tests/test_separable.py',
  'astropy/modeling/tests/test_compound.py',
  'astropy/modeling/separable.py',
  'astropy/coordinates/tests/test_funcs.py',
  'astropy/modeling/tests/test_input.py',
  'astropy/modeling/tests/example_models.py',
  'astropy/modeling/tests/test_models.py',
  'astropy/modeling/tests/test_models_quantities.py',
  'astropy/coordinates/orbital_elements.py',
  'astropy/stats/tests/test_jackknife.py',
  'astropy/modeling/tests/test_polynomial.py',
  'astropy/table/tests/test_operations.py',
  'astropy/coordinates/earth_orientation.py',
  'astropy/coordinates/tests/test_distance.py',
  'astropy/stats/funcs.py'],
 ['docs/conf.py',
  'astropy/conftest.py',
  'astropy/tests/command.py',
  '.pyinstaller/run_astropy_tests.py',
  'astropy/__init__.py',
  'astropy/tests/helper.py',
  'examples/template/example-template.py',
  'conftest.py',
  'astropy/config/tests/test_configs.py',
  'astropy/io/misc/tests/test_pandas.py',
  'docs/conftest.py',
  'setup.p

In [21]:
print("Swebench Semantic Search Results : ")
res = compute_metrics(ground_truth, ss_predictions, [1,2,3,5,10,15])
for key in res:
    print(f"{key} : {res[key]}")

Swebench Semantic Search Results : 
recall@1 : 0.122
precision@1 : 0.134
recall@2 : 0.18866666666666668
precision@2 : 0.19
recall@3 : 0.252
precision@3 : 0.2526666666666667
recall@5 : 0.31716666666666665
precision@5 : 0.31729999999999997
recall@10 : 0.4182619047619048
precision@10 : 0.41836666666666666
recall@15 : 0.49616190476190475
precision@15 : 0.4962


# TF-IDF

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Document vectorization
def vectorize_documents_tfidf(file_contents):
    # Initialize TfidfVectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(file_contents)
    return tfidf_matrix, vectorizer

# Query comparison
def search_tfidf(query, vectorizer, tfidf_matrix, file_names, k):
    query_tfidf = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    # Get file names in descending order of similarity without returning similarity scores
    results = sorted(zip(file_names, cosine_similarities), key=lambda x: x[1], reverse=True)
    ranked_file_names = [file_name for file_name, _ in results[:k]]
    return ranked_file_names

In [ ]:
tf_idf_predictions = []
for idx,row in swebench_df.iterrows():
    repo = row["repo"]
    hashes = ast.literal_eval(row["hashes"])
    embeddings_df = pd.read_csv(get_embeddings_file_path(repo))
    associated_rows = embeddings_df[embeddings_df['hash'].isin(hashes)]
    code_snippets = associated_rows['code_snippet'].to_list()
    file_paths = associated_rows['file_path'].tolist()
    tfidf_matrix, vectorizer = vectorize_documents_tfidf(code_snippets)
    query = f"{row['problem_statement']} {row['hints_text']}"
    res_files = search_tfidf(query, vectorizer, tfidf_matrix, file_paths, 15)
    tf_idf_predictions.append(res_files)
    print(f"Processed row {idx}")

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [ ]:
new_df = swebench_df[['modified_files']].copy()
new_df['predicted_files'] = semantic_search_predictions
new_df.to_csv('/content/drive/My Drive/NLP/Project/data_new/tfidf_res.csv', index=False)

In [ ]:
print("Swebench Tf-Idf Results : ")
res = compute_metrics(ground_truth, tf_idf_predictions, [1,2,3,5,10,15])
for key in res:
    print(f"{key} : {res[key]}")